In [1]:

import seaborn as sns
from matplotlib import pyplot as plt


from dataflow.core.lispress import parse_lispress, lispress_to_program

from collections import Counter

In [11]:
from dataflow.core.program import BuildStructOp, ValueOp, CallLikeOp

def get_frequencies(path):
    fxns = []


    with open(path) as f1:
        for line in f1:
            program = lispress_to_program(parse_lispress(line),0)[0]

            for expr in program.expressions:
                op = expr.op
                if isinstance(op, BuildStructOp):
                    fxns.append(op.op_schema)
                elif isinstance(op, ValueOp):
                    continue
                elif isinstance(op, CallLikeOp):
                    continue
#                     fxns.append(op.name)
                    
    return Counter(fxns)




In [12]:
counts = get_frequencies("/srv/local1/estengel/resources/data/smcalflow.agent.data/train.tgt")






In [8]:
from collections import defaultdict 

def get_accuracies(true_path, pred_path):
    acc_by_fxn = defaultdict(lambda: defaultdict(int))
    with open(true_path) as tf, pred_path as pf:
        for tl, pl in zip(tf, pf): 
            match = tl.strip() == pl.strip() 
            true_prog = lispress_to_program(parse_lispress(tl),0)[0]
            try:
                pred_prog = lispress_to_program(parse_lispress(pl),0)[0]
            except:
                pred_prog = None
            for expr in true_prog.expressions:
                op = expr.op
                if isinstance(op, BuildStructOp):
                    op_name = op.op_schema
                elif isinstance(op, ValueOp):
                    continue
                elif isinstance(op, CallLikeOp):
                    op_name = op.name 
                    
                if match:
                    acc_by_fxn[op_name]["correct"] += 1
                acc_by_fxn[op_name]["total"] += 1
            
    return acc_by_fxn 

In [9]:
accs = get_accuracies("/srv/local1/estengel/resources/data/smcalflow.agent.data/train.tgt")

TypeError: get_accuracies() missing 1 required positional argument: 'pred_path'

In [17]:
import numpy as np 
count_items = counts.items()
count_median = np.median([x[1] for x in count_items])
below_median = [x for x in count_items if x[1] < count_median]
lower_median = np.median([x[1] for x in below_median])
lower_quartile = [x for x in below_median if x[1] < lower_median]
print(len(count_items))
print(len(below_median))

356
178


In [20]:
import json

with open("/srv/local1/estengel/resources/data/smcalflow.agent.data/train_freqs.json", "w") as f1:
    
    json.dump(counts, f1)